In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Adrenocortical_Cancer"
cohort = "GSE143383"

# Input paths
in_trait_dir = "../../input/GEO/Adrenocortical_Cancer"
in_cohort_dir = "../../input/GEO/Adrenocortical_Cancer/GSE143383"

# Output paths
out_data_file = "../../output/preprocess/Adrenocortical_Cancer/GSE143383.csv"
out_gene_data_file = "../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE143383.csv"
out_clinical_data_file = "../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE143383.csv"
json_path = "../../output/preprocess/Adrenocortical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression analysis of metastatic adrenocortical tumors"
!Series_summary	"Background: Adrenocortical carcinoma (ACC) is a rare, often-aggressive neoplasm of the adrenal cortex, with a 14.5-month median overall survival. We asked whether tumors from patients with advanced or metastatic ACC would offer clues as to putative genes that might have critical roles in disease progression or in more aggressive disease biology.   Methods: We conducted comprehensive genomic and expression analyses of 43 ACCs.  Results: Copy number gains and losses matched that previously reported. We identified a median mutation rate of 3.38 per megabase (Mb), somewhat higher than in a previous study possibly related to the more advanced disease. The mutational signature was characterized by a predominance of C>T, C>A and T>C transitions. As in previously reports, only cancer genes TP53 (26%) and beta-catenin (CTNNB1, 14%) were mutated in more than 10% of samples. The T

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Dict, Any, Optional

# 1. Gene Expression Data Availability
# This dataset is about gene expression profiling using Affymetrix PrimeView platform
# and the series summary mentions gene expression analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Looking at the sample characteristics dictionary, we only have gender information
# There's no direct key for adrenocortical cancer trait or age
trait_row = None
age_row = None
gender_row = 0  # Gender is available at key 0

# 2.2 Data Type Conversion
# Since trait_row is None, we don't need to define convert_trait
# But we'll create it with a placeholder function to maintain code structure
def convert_trait(value):
    return None

# Since age_row is None, we don't need to define convert_age
# But we'll create it with a placeholder function to maintain code structure
def convert_age(value):
    return None

def convert_gender(value):
    # Extract value after colon
    if ':' in value:
        gender = value.split(':', 1)[1].strip().lower()
        
        if gender == 'f':
            return 0  # Female
        elif gender == 'm':
            return 1  # Male
        else:
            return None  # Unknown or other
    return None

# 3. Save Metadata
# Trait data availability is determined by whether trait_row is None
is_trait_available = trait_row is not None

# Save the initial filtering information
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we'll skip this substep
# However, we can still extract and save gender data if clinical_data is available
try:
    clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"))
    
    if gender_row is not None:
        # We only have gender data
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=0,  # Using a placeholder since trait data isn't available
            convert_trait=convert_trait,  # Using a placeholder function
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical data:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print("No clinical features are available to extract.")
except Exception as e:
    print(f"Error processing clinical data: {e}")


Error processing clinical data: [Errno 2] No such file or directory: '../../input/GEO/Adrenocortical_Cancer/GSE143383/clinical_data.csv'


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be probe IDs from an Affymetrix microarray
# (format like "11715100_at", "11715101_s_at")
# These are not standard human gene symbols (like BRCA1, TP53) and will need to be mapped

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': [40780.0, 40780.0, 40780.0, 40780.0, 40780.0], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /DEF=g21264570 /REP_O

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain probe IDs and gene symbols
# From the annotation preview, we can see:
# - 'ID' column contains the probe identifiers (e.g., '11715100_at')
# - 'Gene Symbol' column contains the gene symbols (e.g., 'HIST1H3G')

# 2. Create gene mapping DataFrame
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Preview the first few rows and columns of the gene expression data
print("Preview of gene expression data:")
first_genes = gene_data.index[:5]
first_samples = gene_data.columns[:5]
print(gene_data.loc[first_genes, first_samples])


Gene mapping dataframe shape: (49372, 2)
First few rows of gene mapping:
              ID       Gene
0    11715100_at   HIST1H3G
1  11715101_s_at   HIST1H3G
2  11715102_x_at   HIST1H3G
3  11715103_x_at  TNFAIP8L1
4  11715104_s_at      OTOP2


Gene expression data shape after mapping: (19534, 63)
First few gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')
Preview of gene expression data:
       GSM4258059  GSM4258060  GSM4258061  GSM4258062  GSM4258063
Gene                                                             
A1BG      4.86208     5.81829     5.69429     5.99362     6.01689
A1CF      8.17809     7.65680     8.57597     9.20387     9.12301
A2LD1     4.19459     4.34588     4.45649     3.94573     4.17089
A2M      11.48740    10.66540    12.94520    11.30090    12.05390
A2ML1     6.95604     6.78831     7.57040     7.79159     7.42280


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
try:
    # Normalize gene symbols using the NCBI Gene database
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene normalization: {e}")
    # If normalization fails, use the original gene data
    print("Using original gene expression data...")
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Original gene expression data saved to {out_gene_data_file}")

# 2. Create a basic clinical dataframe with gender information
# Since we identified in Step 2 that only gender information is available
print("Creating clinical data with available gender information...")
# First, get the sample identifiers from gene_data columns
sample_ids = gene_data.columns.tolist()

# Create a DataFrame for gender using the clinical data we collected earlier
gender_row = 0  # As identified in Step 2
gender_data = None

try:
    gender_data = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender)
    # Convert to a DataFrame with samples as rows
    clinical_df = gender_data.T
    print(f"Clinical data shape: {clinical_df.shape}")
except Exception as e:
    print(f"Error extracting gender data: {e}")
    # Create an empty DataFrame if gender extraction fails
    clinical_df = pd.DataFrame(index=sample_ids)
    print("Created empty clinical dataframe.")

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Since we don't have trait data (trait_row is None as per Step 2), we can't create a proper linked dataset
# We'll mark the dataset as not usable for trait-gene association studies
print("This dataset doesn't contain trait information for Adrenocortical_Cancer.")

# 4. Validate and save cohort information
note = "Dataset contains gene expression data from adrenocortical tumors, but lacks a proper control group " \
       "and trait classification. Only gender information is available as a clinical feature."

is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,
    is_biased=True,  # Set to True since we don't have a trait to analyze
    df=clinical_df,  # Just pass the clinical data since we don't have a properly linked dataset
    note=note
)

print(f"Dataset usability: {is_usable}")

# 5. Since the dataset isn't usable for trait-gene association studies (no trait data),
# we won't create or save a linked dataset
if is_usable:
    # This condition won't be met based on our assessment, but included for completeness
    # We would need to transpose gene_data and concatenate with clinical_df
    gene_data_t = gene_data.T
    linked_data = pd.concat([clinical_df, gene_data_t], axis=1)
    
    # Handle missing values and check for bias
    linked_data = handle_missing_values(linked_data, trait)
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
    # Save the linked data
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to missing trait information.")

Normalizing gene symbols...


Gene data shape after normalization: (19326, 63)


Normalized gene expression data saved to ../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE143383.csv
Creating clinical data with available gender information...
Clinical data shape: (63, 1)
Clinical data saved to ../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE143383.csv
This dataset doesn't contain trait information for Adrenocortical_Cancer.
Abnormality detected in the cohort: GSE143383. Preprocessing failed.
Dataset usability: False
Dataset is not usable for trait-gene association studies due to missing trait information.
